In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets, transforms
import numpy as np
from matplotlib import pyplot as plt

In [17]:
from EGL import EGLTrainer

In [18]:
import sys
import os

In [19]:
import json

In [20]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from model import SimpleCNN
from BALD.BALD import BALDTrainer
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), 'EGL')))

In [21]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 10           # CIFAR-10
learning_rate = 5e-4

In [22]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [23]:
class NewDataset(Dataset):
    def __init__(self, old_dataset, indexes):
        # Копируем данные на основе указанных индексов
        self.data = [old_dataset[idx] for idx in indexes]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]


In [24]:
TRAIN_LEN = train_dataset.__len__()

In [25]:
INITIAL_TRAIN = 50
p = INITIAL_TRAIN / train_dataset.__len__()
p # Начальный размер датасета

0.001

In [26]:
Trainer = EGLTrainer

In [27]:
res = {}
for p in [0.01, 0.1, 0.2]:
    data_amount = int(TRAIN_LEN * p)
    init_train = data_amount // 10
    initial_indices = np.random.choice(len(train_dataset), size=init_train, replace=False)
    initial_data = Subset(train_dataset, initial_indices)
    unlabeled_indices = list(set(range(len(train_dataset))) - set(initial_indices))
    unlabeled_data = NewDataset(train_dataset, unlabeled_indices)


    train_dataloader = DataLoader(initial_data, batch_size=128, shuffle=True)
    val_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)
    pool_dataloader = DataLoader(unlabeled_data, batch_size=128, shuffle=False)

    print(f" Размер обучающего датасета {len(initial_data)}")
    print(f" Размер AL датасета {len(unlabeled_data)}")
    print(f" Размер тестового датасета {len(test_dataset)}")

    model = SimpleCNN(NUM_CLASSES).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate )
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    trainer = Trainer(model=model.to(DEVICE),
                optimizer=optimizer,
                criterion=criterion, 
                pool_loader=pool_dataloader, 
                train_loader=train_dataloader, 
                val_loader=val_dataloader, 
                device=DEVICE)
    
    _,_,f1 = trainer.fit(end_data_amaunt = data_amount, num_samples = init_train, epochs_for_batch = 5)
    print(f1)
    res[p] = f1
    torch.cuda.empty_cache()
    


 Размер обучающего датасета 50
 Размер AL датасета 49950
 Размер тестового датасета 10000


Training: 100%|██████████| 1/1 [00:00<00:00, 18.20it/s]


Epoch 1/50 - Train data: 50
Train Loss: 2.3094


Training: 100%|██████████| 1/1 [00:00<00:00, 21.74it/s]


Epoch 2/50 - Train data: 50
Train Loss: 2.1753


...

Epoch 50/50 - Train data: 10000
Train Loss: 0.4957


EGL: 100%|██████████| 313/313 [00:01<00:00, 223.42it/s]


40000


Validating: 100%|██████████| 79/79 [00:01<00:00, 41.91it/s]

Validation Loss: 1.1241, Accuracy: 0.6307, F1 Score: 0.6276
0.6276495971734664


In [28]:
def save_f1_to_json(method_name, res, file_path="results.json"):
    # Создаем данные для текущего метода


    # Если файл существует, открываем и загружаем данные
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                data = {}  # Если файл пустой или некорректный
    else:
        data = {}

    # Обновляем или добавляем данные для текущего метода
    data[method_name] = res

    # Записываем обновленные данные обратно в файл
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

In [29]:
res

{0.01: 0.3469231816269731, 0.1: 0.5808840601242585, 0.2: 0.6276495971734664}

In [30]:
method_name = "EGL"  # Имя текущего метода

# Сохраняем данные
save_f1_to_json(method_name, res)